### Dependencies

In [18]:
import cv2
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python import pywrap_tensorflow

sys.path.append('..')

import video_input as vi
import model as model
import utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['videos_folder'] = os.path.join('..', params['videos_folder'])

### Set test image

In [4]:
img_path = os.path.join('..',params['temp_dir'], 'test_img.jpg')
img_width = 224
img_height = 224

test_img = cv2.imread(img_path)
print('Original Size',test_img.shape)

img_resize = cv2.resize(test_img,(img_width,img_height))
img_resize = img_resize[None, ...]

img_normalized = (img_resize / 255.0)
print('Resized and normalized image', img_normalized.shape)

Original Size (2448, 3264, 3)
Resized and normalized image (1, 224, 224, 3)


### Load pretrained model

In [5]:
pretrain_model = model._initialize_pretrained_model()

### Make predictions

In [23]:
video_gen = vi.all_data_videos(params)

NameError: name 'videos' is not defined

In [17]:
block4_pool_features = pretrain_model.predict(next(video_gen))

TypeError: 'NoneType' object is not an iterator

In [8]:
block4_pool_features.shape

(15, 5, 5, 1536)